In [ ]:
#General
import re
import sys
import pprint
import seaborn as sb
import numpy   as np
import pandas  as pd
import random  as rnd

from bios    import read
from os.path import isfile
from copy    import deepcopy
from time    import time
from tqdm    import tqdm
from getdist import plots

from scipy.interpolate import interp1d
from scipy.stats       import multivariate_normal
from itertools         import repeat

from getdist import plots,loadMCSamples,MCSamples

from theory_code.distance_theory import TheoryCalcs

#Plotting
import matplotlib
import matplotlib.pyplot as plt

from matplotlib import rc

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})

red    = '#8e001c'
yellow = '#ffb302'

sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}
bottomlegend = {'bbox_to_anchor': (0.35,-0.2), 
                'loc': "center left",
                'frameon': False,
                'ncols': 3}

# Settings

In [ ]:
fiducial = {'H0': 68.53,
            'omegam': 0.2948,
            'ombh2': 0.02218,
            'epsilon0_EM': 0.1,
            'epsilon0_GW': -0.0,
            'omk': 0.,
            'mnu': 0.06,
            'nnu': 3.,
            'MB': -19.2435}

theory_settings = {'zmin': 0.001,
                   'zmax': 5.,
                   'Nz': 1000,
                   'zdrag': 1060}

zplot = np.linspace(0.01,3,100)

BAO_data = './mock_data/BAOmock_pade'
SN_data  = './mock_data/SNmock_pade'
GW_data = './mock_data/GWmock_pade'

In [ ]:
results = {r'$\Lambda$CDM': {r'DESI': {'root': './chains/BAOmock_LCDM',
                                       'sampler': 'MH',
                                       'DDR_model': 'constant',
                                       'Nchains': 1,
                                       'color': red,
                                       'filled': True},
                             r'Pantheon+$M_B$': {'root': './chains/SNmock_LCDM',
                                                 'sampler': 'MH',
                                                 'DDR_model': 'constant',
                                                 'color': 'purple',
                                                 'Nchains': 1,
                                                 'filled': True},
                            r'ET': {'root': './chains/GWmock_LCDM',
                                                 'DDR_model': 'constant',
                                                  'sampler': 'MH',
                                                 'color': 'forestgreen',
                                                 'Nchains': 1,
                                                 'filled': True},
                             
                             #r'DESI+Pantheon+$M_B$': {'root': './chains/DESI_Pantheon_LCDM',
                             #                         'sampler': 'MH',
                             #                         'DDR_model': 'constant',
                             #                         'color': 'black',
                             #                         'Nchains': 4,
                             #                         'filled': True},
                            }
                               
          }

# Analysis

In [ ]:
def analyze_results(name,info):
    analysis = deepcopy(info)
    
    print('')
    print('\x1b[1;31m Analyzing {} \x1b[0m'.format(name))
    
    chain_info = deepcopy(info)
    if info['sampler'] == 'MH':
        sample = loadMCSamples(info['root'], settings={'ignore_rows': 0.3})
    
        if info['Nchains']>1:
            print('R-1({}) with {:.0f}% of points ignored = {:.3f}'.format(name,30,
                                                                           sample.getGelmanRubin()))
        else:
            print('Single chain, no R-1 computed. Trust Cobaya and hope for the best')
            
        columns = open(info['root']+'.1.txt').readline().rstrip().split()
        columns.pop(0)    
        chains = []
        for ch in range(info['Nchains']):
            chains.append(pd.read_csv(info['root']+'.{}.txt'.format(ch+1),
                                      sep='\s+',skiprows=1,header=None,names=columns))
            
        for par in ['omegam','H0']:
            plt.figure()
            plt.title(name)
            for chain in chains:
                plt.plot(chain.index,chain[par])
            plt.ylabel(par)
            plt.axhline(y=fiducial[par],ls=':',color='black')
            plt.xscale('log')
            

    elif info ['sampler'] == 'Nautilus':
        chain_info['outyaml'] = read(info['root']+'.params.yaml', file_type='yaml')

        primary_pars = {par: chain_info['outyaml']['params'][par]['latex']
                        for par in chain_info['outyaml']['params']
                        if type(chain_info['outyaml']['params'][par]) == dict and 
                        'prior' in chain_info['outyaml']['params'][par]}

        derived_pars = {par: chain_info['outyaml']['params'][par]['latex']
                        for par in chain_info['outyaml']['params']
                        if type(chain_info['outyaml']['params'][par]) == dict and 
                        'prior' not in chain_info['outyaml']['params'][par]}

        pars = primary_pars | derived_pars

        raw_chains = pd.read_csv(info['root']+'_chain.txt',sep='\s+',
                                 header=0)#,names=['weight','minuslogpost']+list(pars.keys()))
        raw_chains['Chain number'] = 1

        chain_info['Nautilus pars']  = pars
        chain_info['Sampled points'] = raw_chains
        chain_info['evidence']       = chain_info['outyaml']['evidence']
        
        sample=MCSamples(samples=chain_info['Sampled points'][list(chain_info['Nautilus pars'].keys())].values,
                         names=list(chain_info['Nautilus pars'].keys()),
                         labels=list(chain_info['Nautilus pars'].values()),label=name)

        sample.root = info['root']

        
    analysis['MCsamples'] = sample
    analysis['bounds']    = sample.getTable(limit=1).tableTex()
    
    all_pars     = sample.getParamNames().list()
    labels       = sample.getParamNames().labels()
    primary_pars = sample.getParamNames().getRunningNames()
    
    analysis['means'] = {par:val for par,val in zip(all_pars,sample.getMeans()) if par in fiducial}
     #MM: theory specific part
    for par in fiducial:
        if par not in analysis['means']:
            analysis['means'][par] = fiducial[par]
            
    case_settings = deepcopy(theory_settings)
    case_settings['DDR_model'] = info['DDR_model']
            
    analysis['theory'] = TheoryCalcs(case_settings,analysis['means'])
    ###############################
    
    
    print(analysis['bounds'])
    if 'evidence' in chain_info:
        print('')
        print('log(Z)={:3f}'.format(chain_info['evidence']))
                               
    all_pars = sample.getParamNames().list()
                               
    analysis['Sampled points']         = pd.DataFrame(sample.makeSingleSamples(),columns=all_pars)
    analysis['Sampled points']['Case'] = name
                    
    return analysis

In [ ]:
analyzed_results = {model: {name: analyze_results(name,resdict) for name,resdict in model_dict.items()} 
                    for model,model_dict in results.items()}

# Parameter plots

## General

In [ ]:
for model,model_dict in analyzed_results.items():
    g = plots.get_subplot_plotter(subplot_size=1,width_inch=12, scaling=False)
    g.settings.figure_legend_frame = False
    g.settings.axes_fontsize=20
    g.settings.axes_labelsize=20
    g.settings.legend_fontsize=20
    g.settings.axis_marker_color = 'black'
    g.settings.axis_marker_ls = '--'
    g.settings.axis_marker_lw = 1
    g.settings.axis_tick_x_rotation = 45
    g.triangle_plot([result['MCsamples'] for result in model_dict.values()], 
        ['omegam','H0','ombh2'],
        filled=[result['filled'] for result in model_dict.values()],
        legend_loc='upper right',
        legend_labels=[result for result in model_dict.keys()],
        contour_colors=[result['color'] for result in model_dict.values()],
        contour_lws=2,
        markers={k:v for k,v in fiducial.items()})    
    plt.suptitle(model)
    g.fig.align_ylabels()
    g.fig.align_xlabels();

In [ ]:
for model, model_dict in analyzed_results.items():
    EM_model_dict = {k: v for k, v in model_dict.items() if k in ['DESI', 'Pantheon+$M_B$']}
    
    g = plots.get_subplot_plotter(subplot_size=1, width_inch=12, scaling=False)
    g.settings.figure_legend_frame = False
    g.settings.axes_fontsize = 20
    g.settings.axes_labelsize = 20
    g.settings.legend_fontsize = 20
    g.settings.axis_marker_color = 'black'
    g.settings.axis_marker_ls = '--'
    g.settings.axis_marker_lw = 1
    g.settings.axis_tick_x_rotation = 45
    
    g.triangle_plot(
        [result['MCsamples'] for result in EM_model_dict.values()], 
        ['epsilon0_EM','omegam', 'H0', 'ombh2'],
        filled=[result['filled'] for result in EM_model_dict.values()],
        legend_loc='upper right',
        legend_labels=list(EM_model_dict.keys()),
        contour_colors=[result['color'] for result in EM_model_dict.values()],
        contour_lws=2,
        markers={k: v for k, v in fiducial.items()}
    )
    
    plt.suptitle(model)
    g.fig.align_ylabels()
    g.fig.align_xlabels()


In [ ]:
for model, model_dict in analyzed_results.items():
    GW_model_dict = {k: v for k, v in model_dict.items() if k in ['ET']}
    
    g = plots.get_subplot_plotter(subplot_size=1, width_inch=12, scaling=False)
    g.settings.figure_legend_frame = False
    g.settings.axes_fontsize = 20
    g.settings.axes_labelsize = 20
    g.settings.legend_fontsize = 20
    g.settings.axis_marker_color = 'black'
    g.settings.axis_marker_ls = '--'
    g.settings.axis_marker_lw = 1
    g.settings.axis_tick_x_rotation = 45
    
    g.triangle_plot(
        [result['MCsamples'] for result in GW_model_dict.values()], 
        ['epsilon0_GW','omegam', 'H0', 'ombh2'],
        filled=[result['filled'] for result in GW_model_dict.values()],
        legend_loc='upper right',
        legend_labels=list(GW_model_dict.keys()),
        contour_colors=[result['color'] for result in GW_model_dict.values()],
        contour_lws=2,
        markers={k: v for k, v in fiducial.items()}
    )
    
    plt.suptitle(model)
    g.fig.align_ylabels()
    g.fig.align_xlabels()


## DDR

In [ ]:
for model, model_dict in analyzed_results.items():
    EM_model_dict = {k: v for k, v in model_dict.items() if k in ['DESI', 'Pantheon+$M_B$']}
    
    g = plots.get_subplot_plotter(subplot_size=1, width_inch=12, scaling=False)
    g.settings.figure_legend_frame = False
    g.settings.axes_fontsize = 20
    g.settings.axes_labelsize = 20
    g.settings.legend_fontsize = 20
    g.settings.axis_marker_color = 'black'
    g.settings.axis_marker_ls = '--'
    g.settings.axis_marker_lw = 1
    g.settings.axis_tick_x_rotation = 45
    
    g.triangle_plot(
        [result['MCsamples'] for result in EM_model_dict.values()], 
        ['epsilon0_EM','omegam'],
        filled=[result['filled'] for result in EM_model_dict.values()],
        legend_loc='upper right',
        legend_labels=list(EM_model_dict.keys()),
        contour_colors=[result['color'] for result in EM_model_dict.values()],
        contour_lws=2,
        markers={k: v for k, v in fiducial.items()}
    )
    
    plt.suptitle(model)
    g.fig.align_ylabels()
    g.fig.align_xlabels()

# Tension plots

## $H_0$

In [ ]:
for model,model_dict in analyzed_results.items():
    g = plots.get_single_plotter(width_inch=5)
    g.plot_1d([result['MCsamples'] for result in model_dict.values()],'H0',
              colors=[result['color'] for result in model_dict.values()], 
              normalized=True,
              lims=[66,76],
              lws=3)
    g.add_x_bands(73.04, 1.04)
    g.add_x_bands(68.53, 0.54)
    g.add_legend([result for result in model_dict.keys()])
    plt.title(model)

## $M_B$

In [ ]:
for model,model_dict in analyzed_results.items():
    if r'DESI+Pantheon' in model_dict:
        g = plots.get_single_plotter(width_inch=5)
        g.plot_1d([model_dict[r'DESI+Pantheon']['MCsamples']],'MB',
                  colors=[model_dict[r'DESI+Pantheon']['color']], 
                  normalized=True,
                  lims=[-19.5,-19.1],
                  lws=3)
        g.add_x_bands(-19.2435 ,0.0373)
        plt.title(model)

## $\epsilon_0^{EM}$ and $\epsilon_0^{GW}$

In [ ]:
for model,model_dict in analyzed_results.items():
     if r'ET' in model_dict:
        g = plots.get_single_plotter(width_inch=5)
        g.plot_1d([model_dict[r'ET']['MCsamples']],'epsilon0_GW',
                  colors=[model_dict[r'ET']['color']], 
                  normalized=True,
                  lims=[-0.2,0.2],
                  lws=3)
        g.add_x_bands(-0.1 ,0.0373)
        plt.title(model)

In [ ]:
for model,model_dict in analyzed_results.items():
    EM_model_dict = {k: v for k, v in model_dict.items() if k in ['DESI', 'Pantheon+$M_B$']}
    g = plots.get_single_plotter(width_inch=5)
    g.plot_1d([result['MCsamples'] for result in EM_model_dict.values()],'epsilon0_EM',
              colors=[result['color'] for result in EM_model_dict.values()], 
              normalized=True,
              lws=3)
    g.add_x_bands(0.1 ,0.0373)
    plt.title(model)

# Theory plots

In [ ]:
dataset_GW = np.load(GW_data+'.npy',allow_pickle=True).item()
dataset_SN = np.load(SN_data+'.npy',allow_pickle=True).item()
dataset_DHDM = np.load(BAO_data+'_DHDM.npy',allow_pickle=True).item()
dataset_DV = np.load(BAO_data+'_DV.npy',allow_pickle=True).item()


In [ ]:
analyzed_results_LCDM=analyzed_results['$\\Lambda$CDM']


In [ ]:
params_DESI= {'omegam':analyzed_results_LCDM['DESI']['means']['omegam'],
            'H0': analyzed_results_LCDM['DESI']['means']['H0'],
            'epsilon0_EM': 0.0,
            'epsilon0_GW': 0.0,
            'ombh2': analyzed_results_LCDM['DESI']['means']['ombh2'],
            'MB': -19.2435,
            'mnu':0.06,
            'nnu':3.}

params_Pantheon = {'omegam':analyzed_results_LCDM['Pantheon+$M_B$']['means']['omegam'],
            'H0': analyzed_results_LCDM['Pantheon+$M_B$']['means']['H0'],
            'epsilon0_EM': analyzed_results_LCDM['Pantheon+$M_B$']['means']['epsilon0_EM'],
            'epsilon0_GW': 0.0,
            'ombh2': analyzed_results_LCDM['Pantheon+$M_B$']['means']['ombh2'],
            'MB': analyzed_results_LCDM['Pantheon+$M_B$']['means']['MB'],
            'mnu':0.06,
            'nnu':3.}

params_ET = {'omegam':analyzed_results_LCDM['ET']['means']['omegam'],
            'H0': analyzed_results_LCDM['ET']['means']['H0'],
            'epsilon0_EM': 0.0,
            'epsilon0_GW': analyzed_results_LCDM['ET']['means']['epsilon0_GW'],
            'ombh2': analyzed_results_LCDM['ET']['means']['ombh2'],
            'MB': -19.2435,
            'mnu':0.06,
            'nnu':3.}

settings = {'zmin': 0.001,
            'zmax': 5.,
            'Nz': 1000,
            'zdrag': 1060,
            'DDR_model': 'padè'}

theory_DESI = TheoryCalcs(settings,params_DESI)
theory_Pantheon = TheoryCalcs(settings,params_Pantheon)
theory_ET = TheoryCalcs(settings,params_ET)

z_camb  = np.linspace(0.001,5.,10000)
theory_padè = {'DESI': {'dA': theory_DESI.dA(z_camb),
                    'Hz' : theory_DESI.Hz(z_camb)*3*1e5,
                    'DH'  :  theory_DESI.DH(z_camb),
                    'DV' : theory_DESI.DV(z_camb),
                    'DM': theory_DESI.DM(z_camb) },
          'Pantheon': {'mB': theory_Pantheon.mB(z_camb),
                       'dL_EM' : theory_Pantheon.DL_EM(z_camb),
                       'Hz' : theory_Pantheon.Hz(z_camb)*3*1e5,
                       'eta_EM': theory_Pantheon.eta_EM(z_camb)},
          
          'ET': {'dL_GW' : theory_ET.DL_GW(z_camb),
                 'Hz' : theory_ET.Hz(z_camb)*3*1e5,
                 'eta_GW': theory_ET.eta_GW(z_camb)},
         
         }


In [ ]:

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['DESI']['Hz'], label="DESI", color='red', linewidth=3, zorder=1)
plt.plot(z_camb, theory_padè['Pantheon']['Hz'], label="Pantheon", color='green', linewidth=3, zorder=1)
plt.errorbar(0, params_Pantheon['H0'], yerr=1.3, fmt='o', color='blue', label=r"$H_0 = 73.0 \pm 1.3$", zorder=2, linewidth=3)

plt.errorbar(0, params_DESI['H0'], yerr=1.2, fmt='o', color='orange', label=r"$H_0 = 69.1 \pm 1.2$", zorder=3, linewidth=3)

plt.xlabel('Redshift (z)')
plt.ylabel('H(z) km/s/Mpc')
plt.xlim(-0.01,1)
plt.ylim(60,130)
plt.title('H(z)  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['ET']['eta_GW'], label="ET", color='blue', linewidth=3, zorder=1)
plt.plot(z_camb, theory_padè['Pantheon']['eta_EM'], label="Pantheon", color='green', linewidth=3, zorder=1)
plt.text(2, 1.00, r"$\eta_{GW}(z)$", color='black', fontsize=12, va='bottom')
plt.text(2, 1.15, r"$\eta_{EM}(z)$", color='black', fontsize=12, va='bottom')

plt.xlabel('Redshift (z)')
plt.ylabel('$\eta(z)$')
plt.title('H(z)  vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:
dataset_DHDM['DH_err'] = [np.sqrt(var) for var in np.diag(dataset_DHDM['covmat'])[:len(dataset_DHDM['DH'])]] 

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['DESI']['DH'], label="DESI", color='red', linewidth=3, zorder=1)
plt.errorbar(dataset_DHDM['z'], dataset_DHDM['DH'], yerr=dataset_DHDM['DH_err'], color='darkslategrey',
             fmt='o', ecolor='grey', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dH')

plt.xlabel('Redshift (z)')
plt.ylabel('DH(z)')
plt.title('DH(z) vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:
dataset_DHDM['DM_err'] = [np.sqrt(var) for var in np.diag(dataset_DHDM['covmat'])[-len(dataset_DHDM['DH']):]] 

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['DESI']['DM'], label="DESI", color='red', linewidth=3, zorder=1)
plt.errorbar(dataset_DHDM['z'], dataset_DHDM['DM'], yerr=dataset_DHDM['DM_err'], color='darkslategrey',
             fmt='o', ecolor='grey', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dH')

plt.xlabel('Redshift (z)')
plt.ylabel('DM(z)')
plt.title('DM(z) vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:
dataset_DV['DV_err'] = [np.sqrt(var) for var in np.diag(dataset_DV['covmat'])] 

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['DESI']['DV'], label="DESI", color='red', linewidth=3, zorder=1)
plt.errorbar(dataset_DV['z'], dataset_DV['DV'], yerr=dataset_DV['DV_err'], color='darkslategrey',
             fmt='o', ecolor='grey', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dH')

plt.xlabel('Redshift (z)')
plt.ylabel('DV(z)')
plt.title('DV(z) vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:
dataset_SN['mB_err'] = [np.sqrt(var) for var in np.diag(dataset_SN['covmat'])] 

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['Pantheon']['mB'], label="Pantheon", color='green', linewidth=3, zorder=1)
plt.errorbar(dataset_SN['z'], dataset_SN['mB'], yerr=dataset_SN['mB_err'], color='darkslategrey',
             fmt='o', ecolor='grey', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dH')

plt.xlabel('Redshift (z)')
plt.ylabel('$m_B$(z)')
plt.title('$m_B$(z) vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()

In [ ]:
dataset_GW['dL_err'] = [np.sqrt(var) for var in np.diag(dataset_GW['covmat'])] 

plt.figure(figsize=(8, 6))

plt.plot(z_camb, theory_padè['ET']['dL_GW'], label="ET", color='blue', linewidth=3, zorder=1)
plt.errorbar(dataset_GW['z'], dataset_GW['dL'], yerr=dataset_GW['dL_err'], color='darkslategrey',
             fmt='o', ecolor='grey', capsize=2, markersize=2, alpha=0.8,  zorder=1, label='Observed dH')

plt.xlabel('Redshift (z)')
plt.ylabel('$d_L^{GW}$(z)')
plt.title('$m_B$(z) vs. z $\epsilon_{EM}=0.1$, $\epsilon_{GW}=0.0$')
plt.legend()
plt.show()